In [ ]:
#RANDOM NUMBERS GENRATOR
np.random.seed(1337) 

import numpy as np
import math

# ~50% correlated ratings
N = int(1e6)
A = np.random.choice(5,size=(N))
B = A*1
ind = np.random.rand(N) >= 0.5
B[ind] = np.random.choice(5,size=(np.sum(ind)))



In [1]:
import numpy as np

def quadratic_kappa(y, t, eps=1e-15):
    # Assuming y and t are one-hot encoded!
    num_scored_items = y.shape[0]
    num_ratings = y.shape[1] 
    ratings_mat = np.tile(np.arange(0, num_ratings)[:, None], 
                        reps=(1, num_ratings))
    print("ratings_mat: ", ratings_mat)
    print("ratings_mat.T: ", ratings_mat.T)

    ratings_squared = (ratings_mat - ratings_mat.T) ** 2
#     print(ratings_squared)    
    weights = ratings_squared / (float(num_ratings) - 1) ** 2
    print(weights)

    # We norm for consistency with other variations.
    y_norm = y / (eps + y.sum(axis=1)[:, None])
#     print(y)

#     print(y_norm)

    # The histograms of the raters.
    hist_rater_a = y_norm.sum(axis=0)
    hist_rater_b = t.sum(axis=0)
    
#     print(hist_rater_a)
#     print(hist_rater_b)

    # The confusion matrix.
    conf_mat = np.dot(y_norm.T, t)
    
      
    print("conf_mat: ",conf_mat)


    # The nominator.
    nom = np.sum(weights * conf_mat)
    expected_probs = np.dot(hist_rater_a[:, None], 
                          hist_rater_b[None, :])
    
#     print(nom)
#     print(expected_probs)

    # The denominator.
    denom = np.sum(weights * expected_probs / num_scored_items)
#     print(denom)

    return 1 - nom / denom
    

    
nb_classes = 5
from keras.utils import np_utils

x = np.array([(1,2) , (4,2), (4,0)])
f = np.array([(1,2) , (4,3), (4,0)])

x = x.flatten()
f = f.flatten()

# x = A 
# f = B 

print("x: ", x)

x = np_utils.to_categorical(x, nb_classes)
f = np_utils.to_categorical(f, nb_classes)

print("x: ", x)


quadratic_kappa(x,f)

Using Theano backend.


('x: ', array([1, 2, 4, 2, 4, 0]))
('x: ', array([[ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 1.,  0.,  0.,  0.,  0.]]))
('ratings_mat: ', array([[0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1],
       [2, 2, 2, 2, 2],
       [3, 3, 3, 3, 3],
       [4, 4, 4, 4, 4]]))
('ratings_mat.T: ', array([[0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4],
       [0, 1, 2, 3, 4]]))
[[ 0.      0.0625  0.25    0.5625  1.    ]
 [ 0.0625  0.      0.0625  0.25    0.5625]
 [ 0.25    0.0625  0.      0.0625  0.25  ]
 [ 0.5625  0.25    0.0625  0.      0.0625]
 [ 1.      0.5625  0.25    0.0625  0.    ]]
('conf_mat: ', array([[ 1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  2.]]))


Using gpu device 0: GeForce GTX 980 Ti (CNMeM is disabled, cuDNN 4007)


0.96202531645569622

In [ ]:
def quad_kappa_loss(y, t, y_pow=1, eps=1e-15):
    num_scored_items = y.shape[0]
    num_ratings = 5
    tmp = T.tile(T.arange(0, num_ratings).reshape((num_ratings, 1)),
                 reps=(1, num_ratings)).astype(theano.config.floatX)
    weights = (tmp - tmp.T) ** 2 / (num_ratings - 1) ** 2

    y_ = y ** y_pow
    y_norm = y_ / (eps + y_.sum(axis=1).reshape((num_scored_items, 1)))

    hist_rater_a = y_norm.sum(axis=0)
    hist_rater_b = t.sum(axis=0)

    conf_mat = T.dot(y_norm.T, t)

    nom = T.sum(weights * conf_mat)
    denom = T.sum(weights * T.dot(hist_rater_a.reshape((num_ratings, 1)),
                                  hist_rater_b.reshape((1, num_ratings))) /
                  num_scored_items.astype(theano.config.floatX))

    return - (1 - nom / denom)

In [3]:
#METHOD WITH NP ARRAYS, FINDING THE LOSS (Doesn't work in the network because it works with tensonrs instead of npArrays)
from keras import backend as K
import numpy as np35100
import math

def kappa(y_true, y_pred): #Receive two np.array()
    
    #Flatten to became the inputs in singles lines of data
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    
    #Var init
    posRatings = 5
    O  = np.zeros((posRatings,posRatings) ,dtype="int")
    W = E = np.zeros((posRatings,posRatings) ,dtype="float32")
    A = np.zeros((posRatings,1) ,dtype="float32")
    B = np.zeros((posRatings,1) ,dtype="float32")
    
    #Kappa calculation
    for i in xrange(posRatings):
        #print(y_true[:]==i)
        A[i] = np.sum((y_true[:]==i))
        #print(A)
        B[i] = np.sum((y_pred[:]==i))
        
        for j in xrange(posRatings):
            O[i,j] = np.sum((y_true[:]==i) & (y_pred[:]==j))
            W[i,j] = math.pow((i-j), 2)/math.pow((posRatings-1), 2)

    print(O)
    print(W)
    E = np.outer(B,A)
    #Normalization
    E = (E*np.sum(O))/np.sum(E)
    print(E)
    k = 1 -(np.sum(W*O))/(np.sum(W*E))

    return k

def kappa_loss(y_true, y_pred):
    return (1 - kappa(y_true, y_pred))

 #input Example array
# y_true = np.array([1,1,4,4,1,0,2])
# y_pred = np.array([1,2,4,3,1,1,2])

# print(y_true)
# print(y_pred)


    
x = np.array([(1,2) , (4,2), (4,0)])
f = np.array([(1,2) , (4,3), (4,0)])

# x = A
# f = B

print(x)
kappa(x,f)

[[1 2]
 [4 2]
 [4 0]]
[[1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 1 1 0]
 [0 0 0 0 0]
 [0 0 0 0 2]]
[[ 0.      0.0625  0.25    0.5625  1.    ]
 [ 0.0625  0.      0.0625  0.25    0.5625]
 [ 0.25    0.0625  0.      0.0625  0.25  ]
 [ 0.5625  0.25    0.0625  0.      0.0625]
 [ 1.      0.5625  0.25    0.0625  0.    ]]
[[ 0.16666667  0.16666667  0.33333334  0.          0.33333334]
 [ 0.16666667  0.16666667  0.33333334  0.          0.33333334]
 [ 0.16666667  0.16666667  0.33333334  0.          0.33333334]
 [ 0.16666667  0.16666667  0.33333334  0.          0.33333334]
 [ 0.33333334  0.33333334  0.66666669  0.          0.66666669]]


0.96202531737254382